In [ ]:
import os
import glob
import librosa
import random
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
from pydub import AudioSegment
from scipy.special import comb
%matplotlib inline


# directories
work_dir = os.getcwd()
sample_dir = os.path.join(work_dir, "samples")
source_dir = os.path.join(work_dir, "audio_data")

if not os.path.exists(sample_dir):
    os.mkdir(sample_dir)


In [ ]:
def LoadAudioFile(wave_names, sounds):            
    files = glob.glob(os.path.join(source_dir, "*.wav"))
    files.sort()
    max_length = 0
    for f in files:
        sound = AudioSegment.from_file(f, format="wav")
        sounds.append(sound)
        name = os.path.basename(f)
        wave_names.append(name[:name.index('.')])
        print("wave: %s loaded %d samples" % (name, len(sound)))
        max_length = max(max_length, len(sound))
    return max_length

def GenerateRandomMixedSamples(wave_names_in, sounds_in, num, length = 6000):
    assert len(wave_names_in) == len(sounds_in)
    src_wav_cnt = len(wave_names_in)
            
    #exlode mixing comnbinatin
    total_combi = 2 ** src_wav_cnt
    for i in range(1, total_combi):
        for j in range(num):
            name = None
            mix = AudioSegment.silent(duration=length * 2)
            max_start = 0
            for k in range(src_wav_cnt):
                if i & (1 << k) != 0:
                    start = random.randrange(0, len(sounds_in[k]))
                    max_start = max(max_start, start)
                    mix = mix.overlay(sounds_in[k], position=start, loop=True)
                    if name == None:
                        name = wave_names[k]
                    else:
                        name += ("-" + wave_names[k])
            # get the last of the samples
            mix = mix[-length:]
            gain = random.randrange(-10.0, 10.0)
            mix = mix.apply_gain(gain)
            mix.export(os.path.join(sample_dir, name) + "_" + str(j) + ".wav", format="wav")
    return

# not used
def GenerateSample(wave_names_in, sounds_in, num_samples):
    assert len(wave_names_in) == len(sounds_in)
    samples_cnt_in = len(wave_names_in)
    for i in range(samples_cnt_in):
        s = sounds_in[i]
        audio_len = len(s)
        repeat_audio = s + s
        for j in range(num_samples):
            start = random.randrange(0, audio_len)
            end = start + audio_len
            new_name = os.path.join(sample_dir, wave_names_in[i] + "-" + str(j) + ".wav")
            #print("new_name: %s(%d), start: %d, end: %d" % (os.path.basename(new_name), len(repeat_audio), start, end))
            sj = repeat_audio[start:end]
            sj.export(new_name, format="wav")
    return

#not used
def ExplodeCombination(wave_names_in, sounds_in):
    wave_names_out = []
    samples_out = []
    total_combi = 2 ** len(wave_names)
    for i in range(1, total_combi):
        name = None
        mix = AudioSegment.silent(duration=max_length)
        for j in range(len(wave_names)):
            if i & (1 << j) != 0:
                mix = mix.overlay(sounds[j])
                if name == None:
                    name = wave_names[j]
                else:
                    name += ("-" + wave_names[j])
        assert name != None
        #rint(name, mix, len(mix))
        wave_names_out.append(name)
        samples_out.append(mix)
        #mix.export(os.path.join(output_dir, name) + ".wav", format="wav")
    return wave_names_out, samples_out

# not tested - probably should read sample from disk again use librosa
def PlotWaves(sound_names,raw_sounds):
    fig = plt.figure(figsize=(25,60))
    i = 1
    for n,f in zip(sound_names,raw_sounds):
        plt.subplot(10,1,i)
        librosa.display.waveplot(np.array(f.raw_data),sr=22050)
        plt.title(n)
        i += 1
    plt.suptitle("Figure 1: Waveplot",x=0.5, y=0.915,fontsize=18)
    plt.show()    
    


In [68]:
# clean current sample
files = glob.glob(os.path.join(sample_dir, "*.wav"))
for f in files:
    os.remove(f)

random.seed()

#wave names and sound files
wave_names = []
sounds = []
max_length = 0
max_length = LoadAudioFile(wave_names, sounds)
print("max_length=%d" % (max_length))
GenerateRandomMixedSamples(wave_names, sounds, 50)